In [36]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

#Get list of teams
week1 = pd.read_csv('Week1.csv')
teams = set(week1['Team'])

#Create empty table
table = DataFrame(columns=['W','L','T','Pts','FPtsF','FPtsA'],
                  index=teams)
table = table.fillna(0)

#Get list of fixtures
fixtures = pd.read_csv('Fixtures.csv')
fixtures = fixtures.set_index('Week')

#Functions to calculate points where position is relevant
def goal_points(player):
    if player.ix['Pos'][0] == 'F':
        return 4*player.ix['G']
    elif player.ix['Pos'][0] == 'M':
        return 5*player.ix['G']
    else:
        return 6*player.ix['G']
    
def goals_against(player):
    if (player.ix['Pos'][0] == 'G') or (player.ix['Pos'][0] == 'D'):
        return (-1)*(player.ix['GA']//2)
    else:
        return 0
    
def clean_sheets(player):
    if player.ix['Pos'][0] == 'G':
        return 5*player.ix['CS']
    elif player.ix['Pos'][0] == 'D':
        return 4*player.ix['CS']
    elif player.ix['Pos'][0] == 'M':
        return player.ix['CS']
    else:
        return 0

#For each week so far --- NEED TO CHANGE THE RANGE BELOW ACCORDINGLY (END IS EXCLUSIVE SO 39 FOR FULL SEASON)
for w in range(1,21):
    
    #Load those stats for the starting players each week
    stats = pd.read_csv('week%s.csv'%w)
    selected = stats.query('GS > 0')
        
    #Get relevant stats and convert to points
    fantasy = selected[['Team','Pos','Player','A','BCC','G','GA','GON',
                       'Int','RC','SOG','CS','TkW','YC','PKM','PKS','Sv',]]
    
    fantasy = fantasy.fillna(0)

    fantasy['Pts'] = 2 + (3*fantasy['A'] + fantasy.apply(goal_points, axis=1) + fantasy.apply(goals_against, axis=1) -
                     2*fantasy['GON'] + fantasy['Int']//3 - 3*fantasy['RC'] + fantasy['SOG']//2 +
                     fantasy.apply(clean_sheets, axis=1) + fantasy['TkW']//3 - fantasy['YC'] - 2*fantasy['PKM'] +
                     5*fantasy['PKS'] + fantasy['Sv']//3 + fantasy['BCC'])
    
    fantasy_sums = fantasy.groupby(['Team']).sum()

    #Get that week's home and away teams
    home = list(fixtures.ix[w,'Home'])
    away = list(fixtures.ix[w,'Away'])

    #For each fixture
    for x in range(0,len(home)):
        
        #Get points by home and away team
        hPts = fantasy_sums['Pts'][home[x]] 
        aPts = fantasy_sums['Pts'][away[x]]
        
        #Update the points for and against
        table['FPtsF'][home[x]] += hPts
        table['FPtsF'][away[x]] += aPts
        table['FPtsA'][away[x]] += hPts
        table['FPtsA'][home[x]] += aPts
        
        #Update WLD records
        if hPts>aPts :
            table['W'][home[x]] += 1
            table['L'][away[x]] += 1
        elif hPts==aPts :
            table['T'][home[x]] += 1
            table['T'][away[x]] += 1
        else :
            table['L'][home[x]] += 1
            table['W'][away[x]] += 1
            
#Add in points and sort
table['Pts'] = 3*table['W'] + table['T']
table.sort_values([('Pts'),('FPtsF')], ascending=[False,False], inplace=True)
    
print table

    

KeyError: 19

,Team,Pos,Player,A,BCC,G,GA,GON,Int,RC,SOG,CS,TkW,YC,PKM,PKS,Sv,Pts
0,MCI,M,"Silva, David - MCI",1,0,1,0,0,0,0,1,1,1,0,0,0,0,11
1,MCI,D,"Kolarov, Aleksandar - MCI",0,0,0,0,0,2,0,1,1,0,0,0,0,0,6
2,ARS,M,"Cazorla, Santiago - ARS",0,0,0,2,0,1,0,1,0,2,0,0,0,0,2
3,BOUR,M,"Surman, Andrew - BOUR",0,0,0,1,0,2,0,0,0,0,0,0,0,0,2
4,ARS,M,"�_zil, Mesut - ARS",0,0,0,2,0,1,0,0,0,2,0,0,0,0,2
5,BOUR,D,"Francis, Simon - BOUR",0,0,0,1,0,1,0,0,0,1,0,0,0,0,2
6,SUN,D,"Kaboul, Younes - SUN",0,0,0,4,0,1,0,1,0,3,1,0,0,0,0
7,MCI,M,Fernandinho - MCI,0,1,0,0,0,1,0,0,1,2,1,0,0,0,3
8,NEW,D,"Haidara, Massadio - NEW",0,0,0,2,0,3,0,0,0,2,0,0,0,0,2
9,ARS,D,"Monreal, Nacho - ARS",0,0,0,2,0,0,0,0,0,0,1,0,0,0,0


/Users/Shruti/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [12]:
fantasy

,Team,Pos,Player,A,BCC,G,GA,GON,Int,RC,SOG,CS,TkW,YC,PKM,PKS,Sv,Pts
0,MCI,M,"Silva, David - MCI",1,0,1,0,0,0,0,1,1,1,0,0,0,0,10.0
1,MCI,D,"Kolarov, Aleksandar - MCI",0,0,0,0,0,2,0,1,1,0,0,0,0,0,2.0
2,ARS,M,"Cazorla, Santiago - ARS",0,0,0,2,0,1,0,1,0,2,0,0,0,0,2.0
3,BOUR,M,"Surman, Andrew - BOUR",0,0,0,1,0,2,0,0,0,0,0,0,0,0,2.0
4,ARS,M,"�_zil, Mesut - ARS",0,0,0,2,0,1,0,0,0,2,0,0,0,0,2.0
5,BOUR,D,"Francis, Simon - BOUR",0,0,0,1,0,1,0,0,0,1,0,0,0,0,1.5
6,SUN,D,"Kaboul, Younes - SUN",0,0,0,4,0,1,0,1,0,3,1,0,0,0,0.0
7,MCI,M,Fernandinho - MCI,0,1,0,0,0,1,0,0,1,2,1,0,0,0,2.0
8,NEW,D,"Haidara, Massadio - NEW",0,0,0,2,0,3,0,0,0,2,0,0,0,0,1.0
9,ARS,D,"Monreal, Nacho - ARS",0,0,0,2,0,0,0,0,0,0,1,0,0,0,1.0


In [19]:
stats = pd.read_csv('week4.csv')

def goal_points(player):
    if player.ix['Pos'][0] == 'F':
        return 4*player.ix['G']
    elif player.ix['Pos'][0] == 'M':
        return 5*player.ix['G']
    else :
        return 6*player.ix['G']
    
def goals_against(player):
    if (player.ix['Pos'][0] == 'G') or (player.ix['Pos'][0] == 'D'):
        return (-1)*(player.ix['GA']/2)
    else:
        return 0

stats['GA'] = stats.apply(goals_against, axis=1)

In [20]:
stats['GA']

0      0
1      0
2      0
3      0
4      0
5      0
6      0
7      0
8     -1
9      0
10     0
11    -1
12     0
13     0
14     0
15    -1
16    -1
17     0
18     0
19     0
20     0
21     0
22     0
23     0
24    -1
25     0
26    -1
27     0
28     0
29    -1
      ..
533    0
534    0
535    0
536    0
537    0
538    0
539    0
540    0
541    0
542    0
543    0
544    0
545    0
546    0
547    0
548    0
549    0
550    0
551    0
552    0
553    0
554    0
555    0
556    0
557    0
558    0
559    0
560    0
561    0
562    0
Name: GA, dtype: int64

In [28]:
5.0//2

2.0

't'